<a href="https://colab.research.google.com/github/lfpbarros/Data-Challenge-Coodesh-20221121/blob/main/Data_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd

In [9]:
titulos_netflix = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Challenge TTT/data/netflix_titles.csv', sep=',', encoding='ISO-8859-1')
# titulos_netflix.tail()

In [10]:
titulos_prime = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Challenge TTT/data/amazon_prime_titles.csv', sep=',')
# titulos_prime.tail()

In [11]:
# Um grande problema ao importar as bases, é que podemos ter ids duplicados, para isso vou incluir uma nova coluna em cada df indicando o streaming ao qual pertence

In [12]:
titulos_netflix['streaming'] = 'Netflix'

In [13]:
titulos_prime['streaming'] = 'Amazon Prime Video'

In [14]:
# Mesclar as bases

In [15]:
titulos = pd.concat([titulos_netflix, titulos_prime])
titulos

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...,Amazon Prime Video
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve...",Amazon Prime Video
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",Amazon Prime Video
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",Amazon Prime Video


In [16]:
# Top 10 atores/atrizes por todos os dados;

In [17]:
# ver o que precisa limpar da series

In [18]:
elenco = titulos['cast'].to_list()
lista_limpar = []
for i in elenco:
  lista = str(i).split(',')
  for ator in lista:
    if len(ator) < 4:
      lista_limpar.append(ator)

In [19]:
set(lista_limpar)

{' ',
 ' 2',
 ' 3',
 ' A',
 ' EL',
 ' JP',
 ' JR',
 ' LB',
 ' MD',
 ' TJ',
 ' 真弓',
 '1',
 'BT',
 'DMX',
 'Dev',
 'GPS',
 'Jai',
 'K K',
 'Kii',
 'L',
 'LB',
 'Lea',
 'MAX',
 'MHD',
 'Mia',
 'Nas',
 'Non',
 'PNL',
 'Sri',
 'TC',
 'TJ',
 'Vir',
 'You',
 'nan'}

In [20]:
# vou criar uma lista com algumas informações que são sujeiras do dataframe
lista_limpar = [' ', ' 2',' 3',' A','1','L','nan']
# meu critério para definir o que é sujeira: ser nan, numérico ou apenas uma letra, abreviações como LB resolvi mante por terem atores que as utilizam

In [21]:
def atores(cast):
  elenco = cast.to_list()
  lista_atores = []
  for i in elenco:
    if i != 'NaN':
      lista = str(i).split(',')
      for ator in lista:
        if ator not in lista_limpar:
          lista_atores.append(ator.strip())
  actors = pd.Series(lista_atores)
  return actors

In [22]:
actors = atores(titulos['cast'])
actors.value_counts()[:10].to_frame().rename(columns={0:'contagem'})

,contagem
Anupam Kher,59
Maggie Binkley,56
Amitabh Bachchan,47
Shah Rukh Khan,46
Nassar,43
Akshay Kumar,41
Naseeruddin Shah,39
Paresh Rawal,39
Danny Trejo,39
Om Puri,35


In [23]:
# Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas;

In [24]:
paises_top_5 = titulos['country'].value_counts()[:5].to_frame().rename(columns={'country':'total_filmes'})
paises_top_5

,total_filmes
United States,3071
India,1201
United Kingdom,447
Japan,247
South Korea,199


In [25]:
netflix_paises = titulos_netflix['country'].value_counts().to_frame().rename(columns={'country':'netflix'})

In [26]:
prime_paises = titulos_prime['country'].value_counts().to_frame().rename(columns={'country':'prime'})

In [27]:
qtd_por_streaming = pd.merge(netflix_paises, prime_paises,how="left", left_index=True, right_index=True)
paises_top_5_por_streaming = pd.merge(paises_top_5, qtd_por_streaming, how='left', left_index=True, right_index=True).fillna(0)

In [28]:
paises_top_5_por_streaming['prime'] = paises_top_5_por_streaming['prime'].astype(int)
paises_top_5_por_streaming = paises_top_5_por_streaming[['netflix', 'prime', 'total_filmes']]
paises_top_5_por_streaming

,netflix,prime,total_filmes
United States,2818,253,3071
India,972,229,1201
United Kingdom,419,28,447
Japan,245,2,247
South Korea,199,0,199


In [29]:
# Mês no qual há mais adições de filmes na plataforma Netflix;

In [30]:
titulos_netflix['date_added'] = pd.to_datetime(titulos_netflix['date_added'], format="%m/%Y", infer_datetime_format=True)

In [31]:
titulos_netflix['month_year'] = titulos_netflix['date_added'].dt.to_period('M')
titulos_netflix['month_year'].value_counts()[:1]

2021-07    257
Freq: M, Name: month_year, dtype: int64

In [32]:
# Quantidade de filmes listados como comédia;

In [33]:
titulos_filmes = titulos.query('type == "Movie"')
titulos_filmes.listed_in.str.contains('Comed').value_counts()

False    10107
True      3838
Name: listed_in, dtype: int64

In [34]:
titulos_filmes[titulos_filmes.listed_in.str.contains('Comed')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,Netflix
13,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,"Klara Castanho, Lucca Picon, JÃºlia Gomes, Mar...",NaN,"September 22, 2021",2021,TV-PG,91 min,"Children & Family Movies, Comedies",When the clever but socially-awkward TetÃª joi...,Netflix
22,s23,Movie,Avvai Shanmughi,K.S. Ravikumar,"Kamal Hassan, Meena, Gemini Ganesan, Heera Raj...",NaN,"September 21, 2021",1996,TV-PG,161 min,"Comedies, International Movies",Newly divorced and denied visitation rights wi...,Netflix
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,Netflix
26,s27,Movie,Minsara Kanavu,Rajiv Menon,"Arvind Swamy, Kajol, Prabhu Deva, Nassar, S.P....",NaN,"September 21, 2021",1997,TV-PG,147 min,"Comedies, International Movies, Music & Musicals",A tangled love triangle ensues when a man fall...,Netflix
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,s9650,Movie,Jack,Francis Ford Coppola,"Robin Williams, Diane Lane, Brian Kerwin, Jenn...",NaN,NaN,1996,PG-13,113 min,Comedy,Robin Williams stars as a 10-year-old in a gro...,Amazon Prime Video
9653,s9654,Movie,Date Night (Extended Cut),Shawn Levy,"Mark Wahlberg, Steve Carell, Taraji P. Henson,...",NaN,NaN,2010,UNRATED,102 min,Comedy,A typical suburban couple's date night becomes...,Amazon Prime Video
9659,s9660,Movie,10 Things I Hate About You,Gil Junger,"Heath Ledger, Julia Stiles, Joseph Gordon-Levi...",NaN,NaN,1999,PG-13,97 min,"Comedy, Drama, Romance","On the first day at his new school, Cameron in...",Amazon Prime Video
9660,s9661,Movie,The Man in the Hat,"John-Paul Davidson, Stephen Warbeck","Ciaran Hinds, Stephen Dillane, Maïwenn",NaN,NaN,2021,13+,96 min,Comedy,The Man in the Hat journeys through France in ...,Amazon Prime Video


In [35]:
titulos['listed_in'].value_counts()

Drama                                               986
Comedy                                              536
Drama, Suspense                                     399
Comedy, Drama                                       377
Dramas, International Movies                        362
                                                   ... 
Dramas, Faith & Spirituality, Sports Movies           1
Children & Family Movies, Classic Movies, Dramas      1
Classic Movies, Horror Movies, Thrillers              1
Action & Adventure, Comedies, Music & Musicals        1
Anime, Comedy, Drama                                  1
Name: listed_in, Length: 1031, dtype: int64

In [36]:
generos = titulos['listed_in'].to_list()
lista_comedia = []
for i in generos:
  lista = str(i).split(',')
  for j in lista:
    if 'Comedy' in j or 'Comedies' in j:
      lista_comedia.append(j.strip())
      pass

len(lista_comedia)
pd.Series(lista_comedia).value_counts()

Comedy                          2099
Comedies                        1674
TV Comedies                      581
Stand-Up Comedy                  343
Stand-Up Comedy & Talk Shows      56
dtype: int64

In [37]:
# Lista de todos os gêneros de filmes;

In [38]:
lista_generos = set(titulos_netflix['listed_in'].to_list())
lista_generos_final = []
for i in lista_generos:
  lista = i.split(',')
  for j in lista:
    lista_generos_final.append(j.strip())

lista_generos_final = set(lista_generos_final)
lista_generos_final

{'Action & Adventure',
 'Anime Features',
 'Anime Series',
 'British TV Shows',
 'Children & Family Movies',
 'Classic & Cult TV',
 'Classic Movies',
 'Comedies',
 'Crime TV Shows',
 'Cult Movies',
 'Documentaries',
 'Docuseries',
 'Dramas',
 'Faith & Spirituality',
 'Horror Movies',
 'Independent Movies',
 'International Movies',
 'International TV Shows',
 "Kids' TV",
 'Korean TV Shows',
 'LGBTQ Movies',
 'Movies',
 'Music & Musicals',
 'Reality TV',
 'Romantic Movies',
 'Romantic TV Shows',
 'Sci-Fi & Fantasy',
 'Science & Nature TV',
 'Spanish-Language TV Shows',
 'Sports Movies',
 'Stand-Up Comedy',
 'Stand-Up Comedy & Talk Shows',
 'TV Action & Adventure',
 'TV Comedies',
 'TV Dramas',
 'TV Horror',
 'TV Mysteries',
 'TV Sci-Fi & Fantasy',
 'TV Shows',
 'TV Thrillers',
 'Teen TV Shows',
 'Thrillers'}

In [39]:
# A frequênica de "TV Show" de todos os dados e comparativamente em relação as duas plataformas

In [40]:
tv_show = titulos.query('type == "TV Show"')
tv_show_por_streaming = tv_show['streaming'].value_counts().to_frame().T
tv_show_por_streaming['total'] = tv_show['streaming'].value_counts().values.sum()
tv_show_por_streaming

,Netflix,Amazon Prime Video,total
streaming,2676,1854,4530


In [41]:
# A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

In [42]:
movies = titulos.query('type == "Movie"')
movies_por_streaming = movies['streaming'].value_counts().to_frame().T
movies_por_streaming['total'] = movies['streaming'].value_counts().values.sum()
movies_por_streaming

,Amazon Prime Video,Netflix,total
streaming,7814,6131,13945


In [43]:
len(movies)

13945